In [1]:
print("최지인")

최지인


In [1]:
from array import array
import reprlib
import math`

In [6]:
class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

## 10.3 프로토콜과 덕 타이핑

- 예제 10.3의 FrenchDeck 클래스는 은 시퀀스 프로토콜을 구현함
- 코드 어디에도 시퀀스 프로토콜을 따른다고 정의한 곳이 없음
- 하지만, 이 클래스가 시퀀스처럼 동작하기 때문에 시퀀스임

## 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

In [16]:
class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    return self._components[index]
    
- 위의 두 함수를 추가하였으니 아래와 같은 연산 수행이 가능

In [18]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [19]:
v1[0], v1[-1]

(3.0, 5.0)

In [20]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

### 10.4.1 슬라이싱의 작동 방식

In [23]:
class MySeq:
    def __getitem__(self, index):
        return index

In [24]:
s = MySeq()

In [26]:
s[1]

1

In [27]:
s[1:4]

slice(1, 4, None)

In [28]:
s[1:4:2]

slice(1, 4, 2)

In [29]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [30]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [34]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [35]:
slice(None, 10, 2)

slice(None, 10, 2)

In [36]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In [38]:
'ABCDE'[-3:]

'CDE'

### 10.4.2 슬라이스를 인식하는 __getitem__()

In [39]:
class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [53]:
v7 = Vector(range(7))

In [54]:
v7[-1]

6.0

In [57]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [58]:
v7[1, 2]

TypeError: Vector indices must be integers

## 10.5 Vector 버전 #3: 동적 속성 접근

In [70]:
class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0<= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [71]:
v = Vector(range(5))
v.x

0.0

In [72]:
v.x = 10
v.x

AttributeError: readonly attribute 'x'